#22 Movie Recommendation System Using SVD

### Importing the basic libraries

In [ ]:
import numpy as np
import pandas as pd

### Importing & Parsing the dataset as ratings and movies details

In [ ]:
ratingData = pd.io.parsers.read_csv('ratings.dat',
    names=['user_id', 'movie_id', 'rating', 'time'],
    engine='python', delimiter='::')
movieData = pd.io.parsers.read_csv('movies.dat',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::')
print(ratingData)

UnicodeDecodeError: ignored

### Create the ratings matrix of shape (m×u)

In [ ]:
ratingMatrix = np.ndarray(
    shape=(np.max(ratingData.movie_id.values), np.max(ratingData.user_id.values)),
    dtype=np.uint8)
ratingMatrix[ratingData.movie_id.values-1, ratingData.user_id.values-1] = ratingData.rating.values
print(ratingMatrix)

[[  5 237 162 ...  48  49   3]
 [ 91  48  59 ...  48  49  98]
 [ 91  48 109 ... 124   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]


### Subtract Mean off - Normalization

In [ ]:
normalizedMatrix = ratingMatrix - np.asarray([(np.mean(ratingMatrix, 1))]).T
print(normalizedMatrix)

[[-4.75178808e+01  1.84482119e+02  1.09482119e+02 ... -4.51788079e+00
  -3.51788079e+00 -4.95178808e+01]
 [ 2.37584437e+01 -1.92415563e+01 -8.24155629e+00 ... -1.92415563e+01
  -1.82415563e+01  3.07584437e+01]
 [ 1.31763245e+01 -2.98236755e+01  3.11763245e+01 ...  4.61763245e+01
  -7.78236755e+01 -7.78236755e+01]
 ...
 [-3.27814570e-02 -3.27814570e-02 -3.27814570e-02 ... -3.27814570e-02
  -3.27814570e-02 -3.27814570e-02]
 [-2.58278146e-02 -2.58278146e-02 -2.58278146e-02 ... -2.58278146e-02
  -2.58278146e-02 -2.58278146e-02]
 [-2.42880795e-01 -2.42880795e-01 -2.42880795e-01 ... -2.42880795e-01
  -2.42880795e-01 -2.42880795e-01]]


### Computing SVD

In [ ]:
A = normalizedMatrix.T / np.sqrt(ratingMatrix.shape[0] - 1)
U, S, V = np.linalg.svd(A)

### Calculate cosine similarity, sort by most similar and return the top N

In [ ]:
def similar(ratingData, movie_id, top_n):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = ratingData[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', ratingData, ratingData)) #Einstein summation |  traditional matrix multiplication and is equivalent to np.matmul(a,b)
    similarity = np.dot(movie_row, ratingData.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity) #Perform an indirect sort along the given axis (Last axis)
    return sort_indexes[:top_n]

### Select k principal components to represent the movies, a movie_id to find recommendations and print the top_n results

In [ ]:
k = 50
movie_id = 2
top_n = 5

sliced = V.T[:, :k] # representative data
indexes = similar(sliced, movie_id, top_n)

print('Recommendations for Movie {0}: \n'.format(
movieData[movieData.movie_id == movie_id].title.values[0]))
for id in indexes + 1:
    print(movieData[movieData.movie_id == id].title.values[0])

<ipython-input-9-a9dc92513cbb>:5: RuntimeWarning: invalid value encountered in divide
  similarity = np.dot(movie_row, ratingData.T) / (magnitude[index] * magnitude)


NameError: ignored